In [ ]:
import pandas as pd
import numpy as np

# DataFrame Calculations

Hoje veremos como criar novas colunas em um DataFrame. Até o momento, já criamos colunas através de condicionais (usando `.loc` ou `np.where`) e através dos métodos `.astype()`, `.map()` e `.fillna()`.

A criação de colunas é extremamente simples: basta lembrarmos que um `DataFrame` se comporta como um dicionário de `Series`! Podemos criar novas colunas como adicionamos chaves à um dicionário: utilizando o operador de *assignment*, `=`.

Para aula de hoje utilizaremos um novo dataset: os dados do artigo *Sleep  in Mammals: Ecological and Constitutional Correlates*, contendo informações sobre o sono e a vida de certos animais.

## Lendo o DataFrame

Vamos iniciar carregando o DataFrame, olhando a documentação e os dados.

Documentação: 
http://lib.stat.cmu.edu/datasets/sleep

In [ ]:
tb_animals = pd.read_csv('http://www.statsci.org/data/general/sleep.txt', sep='\t')

In [ ]:
tb_animals.describe()

In [ ]:
tb_animals.head()

# Calculos com DataFrames

A forma mais simples de criarmos novas colunas é a partir de constantes, listas ou calculos com outras colunas. Vamos ver como realizar cada um desses passos.

## Colunas constantes

Podemos criar um coluna com valor constante simplesmente atribuindo um número à coluna.

In [ ]:
tb_animals['new_column'] = 1

In [ ]:
tb_animals.head()

In [ ]:
tb_animals = tb_animals.drop('new_column', axis = 1)

## Criando colunas com `lists`

Podemos criar uma coluna a partir de uma lista (ou qualquer outro iterável). O Pandas interpretará o iterável como um `Series`, ou seja, cada elemento dele será visto como uma nova linha da nossa tabela. Logo, precisamos que o iterável tenha comprimento igual ao tamanho da nossa tabela.

In [ ]:
tb_animals['id_linha'] = [i for i in range(tb_animals.shape[0])]

In [ ]:
tb_animals['id_linha']

In [ ]:
tb_animals['erro'] = [1,2,3]

## Criando colunas à partir de contas

Podemos utilizar os operadores matemáticos para realizar operações sobre as colunas de um DataSet. A operação será mapeada à cada elemento da coluna - como em vetores do Numpy.

In [ ]:
tb_animals['BrainWt']/1000 

In [ ]:
tb_animals['BrainWt_kg'] = tb_animals['BrainWt']/1000 

In [ ]:
tb_animals.describe()

## Cálculos entre Colunas

Podemos realizar operações entre colunas - da mesma forma que os operadores booleanos (`<`, `>`, `==`, etc) podem ser aplicados sobre uma coluna para criar uma coluna, os operadores matemáticos podem ser usados entre duas colunas para criar novas colunas.

In [ ]:
tb_animals['BrainWt_kg']/tb_animals['BodyWt']

In [ ]:
tb_animals['ratio_brain_body'] = tb_animals['BrainWt_kg']/tb_animals['BodyWt']

In [ ]:
tb_animals['ratio_brain_body'].describe()

In [ ]:
tb_animals[tb_animals['ratio_brain_body']>0.03]

### Operadores Booleanos entre Colunas

Da mesma forma que podemos realizar a comparação de uma coluna com um valor, podemos criar comparações entre colunas:

In [ ]:
tb_animals['ratio_brain_body']>0.01

In [ ]:
tb_animals['Dreaming'] > tb_animals['NonDreaming']

In [ ]:
tb_animals[tb_animals['Dreaming'] > tb_animals['NonDreaming']]

## Usando métodos de `strings` em colunas

A aplicação dos métodos de `str` é um pouco mais complexa, sintaticamente, que a utilização dos operadores: precisamos utilizar um atributo das `Series` para conseguir acessar os métodos.

In [ ]:
tb_animals['Species'].head()

In [ ]:
tb_animals['Species'].lower()

Para acessar os métodos de `strings` vamos utilizar o atributo `.str` das `Series`

In [ ]:
tb_animals['Species'].str.lower()

In [ ]:
tb_animals['lower_species'] = tb_animals['Species'].str.lower()

Além dos métodos básicos de `strings` podemos utilizar funções de REGEX também!. A síntaxe é a mesma: utilizaremos o atributo `.str` para acessar esses métodos.

Vamos começar com o método `.contains()` que retorna um vetor booleano determinando se um padrão foi encontrado ou não em cada linha de nossa coluna. 

In [ ]:
tb_animals['lower_species'].str.contains(r'monk|ape|man|gorilla|baboon|chimpanzee')

In [ ]:
tb_animals['id_primata'] = tb_animals['lower_species'].str.contains(r'monk|ape|man|gorilla|baboon|chimpanzee')

In [ ]:
sum(tb_animals['id_primata'])

Podemos utilizar o método `.findall()` para guardar a informação de qual parte do `string` deu *match* com nosso padrão:

In [ ]:
tb_animals['lista_primata'] = tb_animals['lower_species'].str.findall(r'monk|ape|man|gorilla|baboon|chimpanzee')

In [ ]:
tb_animals.head(10)

O método `.findall()` retorna uma lista: se quisermos transformar essa lista em um string teremos que utilizar o método `.map()`. Vamos começar definindo uma função para selecionar o primeiro elemento de cada lista e utilizar o método `.map()` para aplicar essa função a nossa coluna.

In [ ]:
# EXERCICIO

## Ordenando valores

Podemos utilizar o método `.sort_values()` para ordenar um DataFrame por uma (ou mais) coluna.

In [ ]:
tb_animals.sort_values(by='ratio_brain_body', ascending=False)

Lembrando que os métodos do DataFrame não alteram o objeto original! Se quisermos guardar nosso resultado precisamos faze-lo explicitamente:

In [ ]:
tb_animals = tb_animals.sort_values(by=['Predation', 'ratio_brain_body'], ascending=False)

In [ ]:
tb_animals.head()

## Métodos de agregação entre colunas

Podemos utilizar os métodos de agregação para criar novas colunas: basta mudar o eixo ao longo do qual a operação é realizada!

In [ ]:
tb_animals[['Predation', 'Exposure', 'Danger']].mean(axis=0)

In [ ]:
tb_animals[['Predation', 'Exposure', 'Danger']].mean(axis=1)

In [ ]:
tb_animals['risco'] = tb_animals[['Predation', 'Exposure', 'Danger']].mean(axis=1)

In [ ]:
tb_animals[['Predation', 'Exposure', 'Danger', 'risco']].mean(axis=0)

# Cálculos Condicionais

Podemos utilizar o atributo `.loc` para criar colunas condicionais. Vamos começar com um exemplo simples: criando uma coluna a partir de uma constante.

## Colunas Condicionais constantes

In [ ]:
tb_animals['flag_alto_risco'] = 0

In [ ]:
tb_animals.loc[tb_animals['risco']>=4, 'flag_alto_risco'] = 1

In [ ]:
tb_animals.groupby('flag_alto_risco').mean()

Um atributo muito útil para esse tipo de visualização é o `.T`: ele nos retorna o DataFrame transposto:

In [ ]:
tb_animals.groupby('flag_alto_risco').mean().T

## Colunas Condicionais utilizando operações

In [ ]:
tb_animals['max_risco'] = tb_animals[['Predation', 'Exposure', 'Danger']].max(axis = 1)

In [ ]:
tb_animals.loc[tb_animals['max_risco'] < 5, 'risco_2'] = tb_animals[['Predation', 'Exposure', 'Danger']].mean(axis = 1)
tb_animals.loc[tb_animals['max_risco'] == 5, 'risco_2'] = 5

In [ ]:
tb_animals['flag_alto_risco_2'] = 0
tb_animals.loc[tb_animals['risco_2']>=4, 'flag_alto_risco_2'] = 1

In [ ]:
tb_animals.groupby('flag_alto_risco_2').mean().T

# Quantis

Os quantis são pontos de corte em uma variável numérica que calculados para que uma % das observações esteja abaixo deste ponto. Por exemplo, o quantil 0.5 (50%, ou *mediana*) da variável `BodyWt` é um número tal que 50% das observações tem `BodyWt` abaixo deste número.

Os quantis mais famosos são os **quartis**:

1. 0.25, ou primeiro quartil, onde 25% das observações estão abaixo do quantil;
1. 0.5, ou mediana, onde 50% das observações estão abaixo do quantil;
1. e 0.75, ou terceiro quartil, onde 75% das observações estão abaixo do quantil.

Além disso, muitas vezes usamos os quantis 0.05 e 0.95 para representar os valores mais altos e mais baixos de uma variável.

In [ ]:
tb_animals['BodyWt'].median()

In [ ]:
tb_animals['BodyWt'].quantile(0.5)

In [ ]:
tb_animals['BodyWt'].quantile([0.25, 0.5, 0.75])

Uma utilização comum dos quantis é a **discretização de variáveis continuas**, ou seja, a criação de uma variável categórica (`string`) a partir de uma variável numérica.

In [ ]:
q25 = tb_animals['BodyWt'].quantile(0.25)
q50 = tb_animals['BodyWt'].quantile(0.5)
q75 = tb_animals['BodyWt'].quantile(0.75)
print(q25, q50, q75)

In [ ]:
tb_animals.loc[tb_animals['BodyWt'] >= q75, 'cat_peso'] = 'Pesados'
tb_animals.loc[tb_animals['BodyWt'] < q75, 'cat_peso'] = 'Médios-Pesados'
tb_animals.loc[tb_animals['BodyWt'] < q50, 'cat_peso'] = 'Leves-Médios'
tb_animals.loc[tb_animals['BodyWt'] < q25, 'cat_peso'] = 'Leves'
tb_animals['cat_peso'].value_counts()

## Categorizando dados

A tarefa acima é tão comum que temos uma função específica para *cortar* uma variável numérica de acordo com seus quantis: a `pd.qcut()`

In [ ]:
# Your code here!
tb_animals['BodyWt_Interval'] = pd.qcut(tb_animals['BodyWt'], 4, ['Leves', 'Leves-Médios', 'Médios-Pesados', 'Pesados'])

In [ ]:
tb_animals['BodyWt_Interval'].value_counts()

In [ ]:
tb_animals['BodyWt_Interval'].value_counts(normalize = True)

Os intervalos entre quantis não são uniforme: no exemplo acima a categoria `Leve` tinha animais entre 0 Kg e 0.6 Kg enquanto a `Médios-Pesados` tinha animais entre 3.3 Kg e 48 Kg! Isso acontece pois ao cortamos através de quantis estamos criando intervalos com número de observações uniforme - por consequencia sacrificamos a uniformidade entre intervalos.

Se quisermos *cortar* uma variável em intervalos iguais podemos utilizar a função `pd.cut`:

In [ ]:
tb_animals['cat_risco'] = pd.cut(tb_animals['risco'], 3)

In [ ]:
tb_animals['cat_risco']

# Bonus: 

## Correlation

*Touching statistics*

In [ ]:
tb_animals.corr()